In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

In [3]:
## Load IMDB dataset
max_features = 10000 # vocab size
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 5s 0us/step


In [4]:
max_len=500
X_train=sequence.pad_sequences(X_train,maxlen=max_len)
X_test=sequence.pad_sequences(X_test,maxlen=max_len)
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [5]:
## Train simple RNN
model=Sequential()
model.add(Embedding(max_features,128,input_length=max_len))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.build(input_shape=(None, max_len))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
## Create an instance of Earlystopping callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
earlystopping

In [9]:
## Train the model with early stopping
history =model.fit(
    X_train,y_train,epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping]
)

Epoch 1/10


625/625 [==============================] - 96s 150ms/step - loss: 1711922.5000 - accuracy: 0.5875 - val_loss: 0.6446 - val_accuracy: 0.5994
Epoch 2/10
625/625 [==============================] - 97s 155ms/step - loss: 4.1844 - accuracy: 0.6398 - val_loss: 0.6323 - val_accuracy: 0.6306
Epoch 3/10
625/625 [==============================] - 93s 149ms/step - loss: 0.5778 - accuracy: 0.6829 - val_loss: 0.5994 - val_accuracy: 0.6582
Epoch 4/10
625/625 [==============================] - 94s 150ms/step - loss: 0.5322 - accuracy: 0.7234 - val_loss: 0.5828 - val_accuracy: 0.6730
Epoch 5/10
625/625 [==============================] - 95s 152ms/step - loss: 0.4815 - accuracy: 0.7572 - val_loss: 0.5699 - val_accuracy: 0.7084
Epoch 6/10
625/625 [==============================] - 93s 149ms/step - loss: 0.4764 - accuracy: 0.7628 - val_loss: 0.5683 - val_accuracy: 0.6942
Epoch 7/10
625/625 [==============================] - 110s 176ms/step - loss: 0.4528 - accuracy: 0.7788 - val_loss: 0.5564

In [10]:
model.save('simple_rnn_imdb.h5')

c:\Users\hp\VS Code\RNN\RNN_word\virtual_env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
